# Data Wrangling - Tweet data of WeRateDogs

## Step 1 - Gather Data
The data for this Project is available from 3 different sources:

    1.  The WeRateDogs Twitter archive. This data is provided in the twitter_archive_enhanced.csv file. This file has to be downloaded manually.
    
    2.  The tweet image predictions data.This file is hosted on Udacity's servers and should be downloaded programmatically using the Requests library and the following URL: https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.
    
    3.  Each tweet's retweet count and favorite count.This data is queried using the Twitter API. 

In [1]:
# import libraries
import pandas as pd
import numpy as np
import requests
import os
import json
import tweepy
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Gather data from source 1 : CSV file

archive = pd.read_csv('twitter-archive-enhanced.csv')

In [3]:
# Gather data from source 2 : Download using the URL from web.

url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
response = requests.get(url)

with open('image-predictions.tsv',mode='wb') as file:
    file.write(response.content)

img_pred = pd.read_csv('image-predictions.tsv',delimiter='\t')

In [7]:
# Gather data from source 3 : JSON data from twitter API using tweepy.

consumer_key = ''
consumer_secret = ''
access_token = ''
access_secret = ''

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth)

In [8]:
tweet_ids = list(archive.tweet_id.unique())
tweet_data = []
failed_tweets = []

for tweet in tweet_ids:
    try:
        ranking = tweet_ids.index(tweet)+1
        print(ranking)
        status = api.get_status(tweet,tweet_mode='extended')
        retweet_count = status.retweet_count
        favorite_count = status.favorite_count
        tweet_data.append({'tweet_id': str(tweet),
                        'retweet_count': retweet_count,
                        'favorite_count': favorite_count})
        
    except Exception as e:
        failed_tweets.append(tweet)       

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [45]:
len(failed_tweets)

1477

In [20]:
# writing the json data to file
# When i read the tweet JSON data earlier, i got only 23 failed tweets. So i replaced
# the tweet_json.txt with my earlier file.

with open('tweet_json.txt',mode='w') as file:
    json.dump(tweet_data,file,indent=4)

In [4]:
# reading json data from file and creating a dataframe
with open('tweet_json.txt',mode='r') as read_file:
    json_data = json.load(read_file)

tweet_extn = pd.DataFrame(json_data,columns=['tweet_id','retweet_count','favorite_count'])

## Step 2 - Asses Data

With the data procurred from the 3 sources, it is ready to be assessed both manually and programatically.

### Programmatic Assessment:

In [6]:
archive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [7]:
archive.describe()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,retweeted_status_id,retweeted_status_user_id,rating_numerator,rating_denominator
count,2.356000e+03,7.800000e+01,7.800000e+01,1.810000e+02,1.810000e+02,2356.000000,2356.000000
mean,7.427716e+17,7.455079e+17,2.014171e+16,7.720400e+17,1.241698e+16,13.126486,10.455433
std,6.856705e+16,7.582492e+16,1.252797e+17,6.236928e+16,9.599254e+16,45.876648,6.745237
min,6.660209e+17,6.658147e+17,1.185634e+07,6.661041e+17,7.832140e+05,0.000000,0.000000
25%,6.783989e+17,6.757419e+17,3.086374e+08,7.186315e+17,4.196984e+09,10.000000,10.000000
50%,7.196279e+17,7.038708e+17,4.196984e+09,7.804657e+17,4.196984e+09,11.000000,10.000000
75%,7.993373e+17,8.257804e+17,4.196984e+09,8.203146e+17,4.196984e+09,12.000000,10.000000
max,8.924206e+17,8.862664e+17,8.405479e+17,8.874740e+17,7.874618e+17,1776.000000,170.000000


In [8]:
archive.duplicated().any()

False

In [9]:
archive[archive['tweet_id'].duplicated()]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo


In [10]:
archive.head(5)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


In [11]:
img_pred.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [12]:
img_pred.describe()

,tweet_id,img_num,p1_conf,p2_conf,p3_conf
count,2.075000e+03,2075.000000,2075.000000,2.075000e+03,2.075000e+03
mean,7.384514e+17,1.203855,0.594548,1.345886e-01,6.032417e-02
std,6.785203e+16,0.561875,0.271174,1.006657e-01,5.090593e-02
min,6.660209e+17,1.000000,0.044333,1.011300e-08,1.740170e-10
25%,6.764835e+17,1.000000,0.364412,5.388625e-02,1.622240e-02
50%,7.119988e+17,1.000000,0.588230,1.181810e-01,4.944380e-02
75%,7.932034e+17,1.000000,0.843855,1.955655e-01,9.180755e-02
max,8.924206e+17,4.000000,1.000000,4.880140e-01,2.734190e-01


In [13]:
img_pred.duplicated().any()

False

In [14]:
img_pred.sample(5)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
1813,833479644947025920,https://pbs.twimg.com/media/C5EdT4jWEAARv2C.jpg,3,golden_retriever,0.727039,True,cocker_spaniel,0.071140,True,Tibetan_mastiff,0.048694,True
1443,775729183532220416,https://pbs.twimg.com/media/CsPxk85XEAAeMQj.jpg,1,web_site,0.989407,False,hand-held_computer,0.002139,False,menu,0.002115,False
1227,745314880350101504,https://pbs.twimg.com/media/Clfj6RYWMAAFAOW.jpg,2,ice_bear,0.807762,False,great_white_shark,0.027040,False,fountain,0.022052,False
456,674774481756377088,https://pbs.twimg.com/media/CV1HztsWoAAuZwo.jpg,1,Chihuahua,0.407016,True,French_bulldog,0.309978,True,Siamese_cat,0.227677,False
1913,854010172552949760,https://pbs.twimg.com/media/C9oNt91WAAAFSLS.jpg,1,English_springer,0.354733,True,collie,0.177538,True,Border_collie,0.131706,True


In [15]:
tweet_extn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2333 entries, 0 to 2332
Data columns (total 3 columns):
tweet_id          2333 non-null object
retweet_count     2333 non-null int64
favorite_count    2333 non-null int64
dtypes: int64(2), object(1)
memory usage: 54.8+ KB


In [16]:
tweet_extn.describe()

,retweet_count,favorite_count
count,2333.000000,2333.000000
mean,2769.305186,7696.594085
std,4687.141638,11946.684150
min,1.000000,0.000000
25%,557.000000,1336.000000
50%,1298.000000,3350.000000
75%,3220.000000,9452.000000
max,79540.000000,159430.000000


In [17]:
tweet_extn.sample(5)

,tweet_id,retweet_count,favorite_count
2226,669749430875258880,65,259
242,844973813909606400,3233,15263
730,779377524342161408,3222,9129
1892,757400162377592832,7039,15522
1459,679862121895714818,630,2485


In [18]:
columns = list(archive.columns)+list(img_pred.columns)+list(tweet_extn.columns)
columns

['tweet_id',
 'in_reply_to_status_id',
 'in_reply_to_user_id',
 'timestamp',
 'source',
 'text',
 'retweeted_status_id',
 'retweeted_status_user_id',
 'retweeted_status_timestamp',
 'expanded_urls',
 'rating_numerator',
 'rating_denominator',
 'name',
 'doggo',
 'floofer',
 'pupper',
 'puppo',
 'tweet_id',
 'jpg_url',
 'img_num',
 'p1',
 'p1_conf',
 'p1_dog',
 'p2',
 'p2_conf',
 'p2_dog',
 'p3',
 'p3_conf',
 'p3_dog',
 'tweet_id',
 'retweet_count',
 'favorite_count']

### Visual Assessment:

Saving the 3 dataframes in CSV format , to be opened in a excel document for visual assessment.

In [19]:
archive.to_csv('archive.csv',index=False)

In [20]:
img_pred.to_csv('img_pred.csv',index=False)

### Observations:

**Quality Issues:**

tweet_extn dataframe:

1. Change tweet_id to int.

archive dataframe:

2. Change timestamp column to datatype Date.
3. Fix issues with names. example: a, all etc. all names starting with a          lowercase letter are probably wrong and should be replaced with null.
4. Change rating numerator to float, beacause the ratings are given in decimals    in some instances.
5. Fix rating numerator. 
      - Currently only the part after the decimal point is considered.
      - Correct the ratings for Text that has slash values like 24/7. These are         mistaken as ratings.
      - In case of multiple dogs in a pic , the rating is given as an                   aggregation. Average out these values, so that the rating corresponds           to a denominator of 10.
6. Fix rating denominator.
7. Remove HTML tags from source column.
8. Drop rows with null ratings.
9. Remove duplicates from expanded URLs column.
10. Fix dog categories that are combined together. When there is more than one dog in a picture, the category is given as a combination of two.

**Tidiness Issues:**
1. Drop columns in_reply_to_status_id, in_reply_to_user_id, retweeted_status_id, retweeted_status_user_id, retweeted_status_timestamp from archive.
2. Merge doggo,pupper,puppo,floffer into a single column called dog category.
3. Merge the 3 dataframes together into a single dataframe.

## Step 3 - Clean

Perform the required actions to clean the data.

In [5]:
# make a copy of data
archive_clean = archive.copy()
img_pred_clean = img_pred.copy()
tweet_extn_clean = tweet_extn.copy()

### Quality Issues

`Define` 

**1.** Change the tweet_id to int in tweet_extn_clean df using astype function.

``Code``

In [6]:
tweet_extn_clean.tweet_id = tweet_extn_clean.tweet_id.astype(int)

``Test``

In [7]:
type(tweet_extn_clean.tweet_id[0])

numpy.int64

``Define``

**2.** Change the datatype of timestamp column in archive_clean df to Datetime using to_datetime function.

``Code``

In [8]:
archive_clean.timestamp = pd.to_datetime(archive_clean['timestamp'])

``Test``

In [9]:
archive_clean.timestamp[0:5]

0   2017-08-01 16:23:56
1   2017-08-01 00:17:27
2   2017-07-31 00:18:03
3   2017-07-30 15:58:51
4   2017-07-29 16:00:24
Name: timestamp, dtype: datetime64[ns]

``Define``

**3.** Replace all names that start with a lowercase letter with None.

``Code``

In [10]:
import string

for index, row in archive_clean.iterrows():
    if (row['name'][0]) in string.ascii_lowercase:
        archive_clean.replace(row['name'],np.nan,inplace=True)

``Test``

In [11]:
archive_clean.name.value_counts().sort_index()

Abby            2
Ace             1
Acro            1
Adele           1
Aiden           1
Aja             1
Akumi           1
Al              1
Albert          2
Albus           2
Aldrick         1
Alejandro       1
Alexander       1
Alexanderson    1
Alf             1
Alfie           5
Alfy            1
Alice           2
Amber           1
Ambrose         1
Amy             1
Amélie          1
Anakin          2
Andru           1
Andy            1
Angel           1
Anna            1
Anthony         1
Antony          1
Apollo          1
               ..
Walker          1
Wallace         3
Wally           2
Walter          4
Watson          2
Wesley          1
Wiggles         1
Willem          1
William         1
Willie          1
Willow          1
Willy           1
Wilson          3
Winifred        1
Winnie          4
Winston         9
Wishes          1
Wyatt           3
Yoda            1
Yogi            2
Yukon           1
Zara            1
Zeek            1
Zeke            3
Zeus      

``Define``

**4.** Change the rating numerator column to float, because the rating numerator has decimals sometimes. Change this using astype function.

``Code``

In [12]:
archive_clean.rating_numerator = archive_clean.rating_numerator.astype(float)

``Test``

In [13]:
type(archive_clean.rating_numerator[0])

numpy.float64

``Define``

**5.** Fix the rating numerator, I have identified individual cases through visual assessment using Excel spreadsheet app.

``Code``

In [14]:
# Fixing the ratings.

column_name = 'rating_numerator'

archive_clean.loc[2335, column_name] = 9
archive_clean.loc[2260, column_name] = -5
archive_clean.loc[1662, column_name] = 10
archive_clean.loc[1662, 'rating_denominator'] = 10
archive_clean.loc[2272, column_name] = 7.5
archive_clean.loc[784, column_name] = 14
archive_clean.loc[2272, 'rating_denominator'] = 10
archive_clean.loc[55, column_name] = 13
archive_clean.loc[313, column_name] = 13
archive_clean.loc[313, 'rating_denominator'] = 10
archive_clean.loc[340, column_name] = 9.75
archive_clean.loc[433, column_name] = 12
archive_clean.loc[433, 'rating_denominator'] = 10
archive_clean.loc[516, column_name] = np.nan
archive_clean.loc[695, column_name] = 9.75
archive_clean.loc[763, column_name] = 11.27
archive_clean.loc[902, column_name] = 11
archive_clean.loc[902, 'rating_denominator'] = 10
archive_clean.loc[1120, column_name] = 12
archive_clean.loc[1120, 'rating_denominator'] = 10
archive_clean.loc[1202, column_name] = 11
archive_clean.loc[1202, 'rating_denominator'] = 10
archive_clean.loc[1228, column_name] = 9.9
archive_clean.loc[1228, 'rating_denominator'] = 10
archive_clean.loc[1254, column_name] = 8
archive_clean.loc[1254, 'rating_denominator'] = 10
archive_clean.loc[1351, column_name] = np.nan
archive_clean.loc[1351, 'rating_denominator'] = np.nan
archive_clean.loc[1433, column_name] = np.nan
archive_clean.loc[1433, 'rating_denominator'] = np.nan
archive_clean.loc[1634, column_name] = 11
archive_clean.loc[1634, 'rating_denominator'] = 10
archive_clean.loc[1635, column_name] = 11
archive_clean.loc[1635, 'rating_denominator'] = 10
archive_clean.loc[1663, column_name] = np.nan
archive_clean.loc[1663, 'rating_denominator'] = np.nan
archive_clean.loc[1712, column_name] = 11.2
archive_clean.loc[1779, column_name] = 12
archive_clean.loc[1779, 'rating_denominator'] = 10
archive_clean.loc[1843, column_name] = 11
archive_clean.loc[1843, 'rating_denominator'] = 10
archive_clean.loc[2074, column_name] = np.nan


In [15]:
# rows with high ratings

archive_clean.loc[188, column_name] = np.nan
archive_clean.loc[189, column_name] = np.nan
archive_clean.loc[290, column_name] = np.nan
archive_clean.loc[979, column_name] = np.nan
archive_clean.loc[1274, column_name] = np.nan

``Test``


In [16]:
archive_clean.rating_numerator.value_counts().sort_index()

-5.00       1
 0.00       2
 1.00       8
 2.00       9
 3.00      19
 4.00      17
 5.00      36
 6.00      32
 7.00      53
 7.50       1
 8.00     103
 9.00     158
 9.75       2
 9.90       1
 10.00    462
 11.00    469
 11.20      1
 11.27      1
 12.00    561
 13.00    353
 14.00     55
 15.00      2
Name: rating_numerator, dtype: int64

``Define``

**6.** Fix rating denominator. The rating denominator has to be 10 for all tweets. Identify all the rating denominator values that are not 10 and correct them using loc function to point to a location.

``Code``

In [17]:
archive_clean.query('rating_denominator != 10')

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
342,832088576586297345,8.320875e+17,3.058208e+07,2017-02-16 04:45:50,"<a href=""http://twitter.com/download/iphone"" r...",@docmisterio account started on 11/15/15,NaN,NaN,NaN,NaN,11.0,15.0,None,None,None,None,None
516,810984652412424192,NaN,NaN,2016-12-19 23:06:23,"<a href=""http://twitter.com/download/iphone"" r...",Meet Sam. She smiles 24/7 &amp; secretly aspir...,NaN,NaN,NaN,"https://www.gofundme.com/sams-smile,https://tw...",NaN,7.0,Sam,None,None,None,None
784,775096608509886464,NaN,NaN,2016-09-11 22:20:06,"<a href=""http://twitter.com/download/iphone"" r...","RT @dog_rates: After so many requests, this is...",7.403732e+17,4.196984e+09,2016-06-08 02:41:38 +0000,https://twitter.com/dog_rates/status/740373189...,14.0,11.0,None,None,None,None,None
1068,740373189193256964,NaN,NaN,2016-06-08 02:41:38,"<a href=""http://twitter.com/download/iphone"" r...","After so many requests, this is Bretagne. She ...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/740373189...,9.0,11.0,None,None,None,None,None
1165,722974582966214656,NaN,NaN,2016-04-21 02:25:47,"<a href=""http://twitter.com/download/iphone"" r...",Happy 4/20 from the squad! 13/10 for all https...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/722974582...,4.0,20.0,None,None,None,None,None
1274,709198395643068416,NaN,NaN,2016-03-14 02:04:08,"<a href=""http://twitter.com/download/iphone"" r...","From left to right:\nCletus, Jerome, Alejandro...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/709198395...,NaN,50.0,None,None,None,None,None
1351,704054845121142784,NaN,NaN,2016-02-28 21:25:30,"<a href=""http://twitter.com/download/iphone"" r...",Here is a whole flock of puppers. 60/50 I'll ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/704054845...,NaN,NaN,NaN,None,None,None,None
1433,697463031882764288,NaN,NaN,2016-02-10 16:51:59,"<a href=""http://twitter.com/download/iphone"" r...",Happy Wednesday here's a bucket of pups. 44/40...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/697463031...,NaN,NaN,None,None,None,None,None
1598,686035780142297088,6.860340e+17,4.196984e+09,2016-01-10 04:04:10,"<a href=""http://twitter.com/download/iphone"" r...",Yes I do realize a rating of 4/20 would've bee...,NaN,NaN,NaN,NaN,4.0,20.0,None,None,None,None,None
1663,682808988178739200,6.827884e+17,4.196984e+09,2016-01-01 06:22:03,"<a href=""http://twitter.com/download/iphone"" r...","I'm aware that I could've said 20/16, but here...",NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None


In [18]:
archive_clean.loc[342, 'rating_denominator'] = np.nan
archive_clean.loc[784, 'rating_denominator'] = 10
archive_clean.loc[1068, 'rating_denominator'] = 10
archive_clean.loc[1068, 'rating_numerator'] = 14
archive_clean.loc[1165, 'rating_denominator'] = 10
archive_clean.loc[1165, 'rating_numerator'] = 13
archive_clean.loc[1274, 'rating_denominator'] = 10
archive_clean.loc[1274, 'rating_numerator'] = 9
archive_clean.loc[1598, 'rating_denominator'] = np.nan
archive_clean.loc[2335, 'rating_denominator'] = 10
archive_clean.loc[516, 'rating_denominator'] = np.nan

``Test``

In [19]:
archive_clean.rating_denominator.value_counts()

10.0    2350
Name: rating_denominator, dtype: int64

``Define``

**7.** Remove the HTML tags from the source column. Use the str.split to split the values.

``Code``

In [20]:
# removing the first tag
archive_clean['new'] = archive_clean['source'].str.split('>',n=1,expand=True)[1]

In [21]:
# removing the second tag
archive_clean['source'] = archive_clean['new'].str.split('<',n=1,expand=True)[0]

In [22]:
#Drop the new colunm
archive_clean.drop(columns='new',inplace=True)

``Test``

In [23]:
archive_clean.source.unique()

array(['Twitter for iPhone', 'Twitter Web Client', 'Vine - Make a Scene',
       'TweetDeck'], dtype=object)

``Define``

8. Drop rows with null ratings. Drop nulls using dropna function for both numerator and denominator columns.

``Code``

In [24]:
archive_clean.dropna(subset=['rating_numerator','rating_denominator'],inplace=True)

``Test``

In [25]:
archive_clean.shape

(2345, 17)

In [26]:
archive_clean.rating_numerator.isnull().any()

False

In [27]:
archive_clean.rating_denominator.isnull().any()

False

In [28]:
type(archive_clean.rating_denominator[0])

numpy.float64

In [30]:
#Changing rating denominator back to int.
archive_clean.rating_denominator = archive_clean.rating_denominator.astype(int)

``Define``

**9.** Drop duplicates from expanded urls column. This can be done using drop_duplicates function

``Code``

In [31]:
archive_clean.drop_duplicates(subset=['expanded_urls'],keep='last',inplace=True)

``Test``

In [32]:
archive_clean[archive_clean.expanded_urls.duplicated()]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo


In [33]:
archive_clean.shape

(2214, 17)

### Tidiness Issues

``Define``

**1:** Drop columns in_reply_to_status_id, in_reply_to_user_id, retweeted_status_id, retweeted_status_user_id, retweeted_status_timestamp from archive. I will not be using these columns for my analysis.

``Code``

In [34]:
archive_clean.drop(columns=['in_reply_to_status_id','in_reply_to_user_id',
                            'retweeted_status_id','retweeted_status_user_id',
                            'retweeted_status_timestamp'],axis=1,inplace=True)

``Test``

In [35]:
archive_clean.columns

Index(['tweet_id', 'timestamp', 'source', 'text', 'expanded_urls',
       'rating_numerator', 'rating_denominator', 'name', 'doggo', 'floofer',
       'pupper', 'puppo'],
      dtype='object')

``Define``

**2.** Merge doggo, pupper, puppo, floofer into a single column called category.

``Code``

In [36]:
archive_clean.replace('None',np.nan,inplace=True)
archive_clean['dog_category'] = archive_clean[archive_clean.columns[8:]].apply(lambda row : ''.join(row.dropna().astype(str)),axis=1)

In [37]:
archive_clean.dog_category.value_counts()

                1858
pupper           232
doggo             76
puppo             25
doggopupper       12
floofer            9
doggofloofer       1
doggopuppo         1
Name: dog_category, dtype: int64

In [38]:
# Drop columns doggo, pupper, puppo, floofer
archive_clean.drop(columns=['doggo','floofer','pupper','puppo'],axis=1,inplace=True)

In [39]:
# Change datatype of category column to category
archive_clean.dog_category = archive_clean.dog_category.astype('category')

In [40]:
#checking if all the datatypes are correct.
archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2214 entries, 0 to 2355
Data columns (total 9 columns):
tweet_id              2214 non-null int64
timestamp             2214 non-null datetime64[ns]
source                2214 non-null object
text                  2214 non-null object
expanded_urls         2213 non-null object
rating_numerator      2214 non-null float64
rating_denominator    2214 non-null int64
name                  1442 non-null object
dog_category          2214 non-null category
dtypes: category(1), datetime64[ns](1), float64(1), int64(2), object(4)
memory usage: 158.2+ KB


``Define`` Quality issue:

10. Fix dog categories overlap. When there is more than one dog in a picture, 
    the category is given as a combination of two. Drop these rows.

``Code``

In [41]:
archive_clean = archive_clean[archive_clean.dog_category != 'doggopuppo']
archive_clean = archive_clean[archive_clean.dog_category != 'doggofloofer']
archive_clean = archive_clean[archive_clean.dog_category != 'doggopupper']

``Test``

In [42]:
archive_clean['dog_category'].unique()

[, doggo, puppo, pupper, floofer]
Categories (5, object): [, doggo, puppo, pupper, floofer]

In [43]:
archive_clean.shape

(2200, 9)

In [44]:
archive_clean = archive_clean.merge(tweet_extn_clean,how='inner',left_on='tweet_id',right_on='tweet_id')

In [45]:
twitter_master = archive_clean.merge(img_pred_clean,how='inner',left_on='tweet_id',right_on='tweet_id')

In [46]:
twitter_master.columns

Index(['tweet_id', 'timestamp', 'source', 'text', 'expanded_urls',
       'rating_numerator', 'rating_denominator', 'name', 'dog_category',
       'retweet_count', 'favorite_count', 'jpg_url', 'img_num', 'p1',
       'p1_conf', 'p1_dog', 'p2', 'p2_conf', 'p2_dog', 'p3', 'p3_conf',
       'p3_dog'],
      dtype='object')

In [100]:
twitter_master.to_csv('twitter_archive_master.csv',index=False)

In [47]:
twitter_master.head()

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,dog_category,retweet_count,...,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,892420643555336193,2017-08-01 16:23:56,Twitter for iPhone,This is Phineas. He's a mystical boy. Only eve...,https://twitter.com/dog_rates/status/892420643...,13.0,10,Phineas,,7858,...,1,orange,0.097049,False,bagel,0.085851,False,banana,0.076110,False
1,892177421306343426,2017-08-01 00:17:27,Twitter for iPhone,This is Tilly. She's just checking pup on you....,https://twitter.com/dog_rates/status/892177421...,13.0,10,Tilly,,5831,...,1,Chihuahua,0.323581,True,Pekinese,0.090647,True,papillon,0.068957,True
2,891815181378084864,2017-07-31 00:18:03,Twitter for iPhone,This is Archie. He is a rare Norwegian Pouncin...,https://twitter.com/dog_rates/status/891815181...,12.0,10,Archie,,3853,...,1,Chihuahua,0.716012,True,malamute,0.078253,True,kelpie,0.031379,True
3,891689557279858688,2017-07-30 15:58:51,Twitter for iPhone,This is Darla. She commenced a snooze mid meal...,https://twitter.com/dog_rates/status/891689557...,13.0,10,Darla,,8034,...,1,paper_towel,0.170278,False,Labrador_retriever,0.168086,True,spatula,0.040836,False
4,891327558926688256,2017-07-29 16:00:24,Twitter for iPhone,This is Franklin. He would like you to stop ca...,https://twitter.com/dog_rates/status/891327558...,12.0,10,Franklin,,8682,...,2,basset,0.555712,True,English_springer,0.225770,True,German_short-haired_pointer,0.175219,True


## Data Analysis

**What are the most frequent dog names recorded?**

In [115]:
twitter_master.name.value_counts().head(10)

Cooper     10
Oliver     10
Tucker     10
Penny      10
Charlie    10
Lucy        9
Lola        8
Bo          8
Sadie       8
Winston     8
Name: name, dtype: int64

**Most tweeted dog breeds**

In [56]:
twitter_master[twitter_master.p1_dog == True]['p1'].value_counts().head(10)

golden_retriever      143
Labrador_retriever     96
Pembroke               88
Chihuahua              81
pug                    57
chow                   44
Samoyed                42
Pomeranian             38
toy_poodle             38
cocker_spaniel         30
Name: p1, dtype: int64

**Most liked dog breed**

In [68]:
twitter_master[twitter_master.p1_dog == True][['p1','favorite_count']].groupby('p1').sum().sort_values('favorite_count',ascending=False).head(10)

,favorite_count
p1,
golden_retriever,1528318
Labrador_retriever,1001182
Pembroke,946131
Chihuahua,656273
Samoyed,488861
French_bulldog,445111
chow,386901
pug,297538
cocker_spaniel,293850


In [69]:
twitter_master[twitter_master.p1_dog == True][['p1','retweet_count']].groupby('p1').sum().sort_values('retweet_count',ascending=False).head(10)

,retweet_count
p1,
golden_retriever,482508
Labrador_retriever,371199
Pembroke,253718
Chihuahua,224572
Samoyed,182396
chow,119082
cocker_spaniel,110117
French_bulldog,109418
pug,100432
